In [1]:
import os
import io
import tweepy
from google.cloud import bigquery
from google.oauth2 import service_account
import pandas_gbq

### Connect to GBQ

In [2]:
# Do our imports for the code
from google.cloud import bigquery
from google.oauth2 import service_account

# Set authentication info
service_path = "/Users/natebender/Desktop/repo/footprints/footprints_audience/"
service_file = 'footprintsrunning-d15e651db9a6.json' # change this to your authentication information  
gbq_proj_id = 'footprintsrunning' # change this to your poroject. 

# And this should stay the same. 
private_key = service_path + service_file

# Now we pass in our credentials so that Python has permission to access our project.
credentials = service_account.Credentials.from_service_account_file(service_path + service_file)

# And finally we establish our connection
gbqclient = bigquery.Client(credentials = credentials, project=gbq_proj_id)

# Set table ID variable
table_id = 'footprintsrunning.twitter_scraping.trail_running'

In [3]:
# Test connection
clean_files = "/Users/natebender/Desktop/repo/footprints/footprints_audience/data"
os.listdir(clean_files)

['.DS_Store', 'old', '20221101_000000_trailrunningtweets.csv']

### Connect to Tweepy

In [4]:
# Set connection to Tweepy

# From the "project set-up" experience
api_key = 'e9hKR9ngqwXalms6q4IndwoAz'
api_key_secret = 'mQzPQWyQvO3ikOqMGjJfKpbFTMcSJKNE9vvywpuSi1Jv0HAjGt'
bearer_token = 'AAAAAAAAAAAAAAAAAAAAADMxgwEAAAAAcEBEj8u5Qy'  

In [5]:
tweepyclient = tweepy.Client(bearer_token, wait_on_rate_limit=True)

In [6]:
# You should have data in the file that you can see via the console. Let's test it here.
#query = (
#    "SELECT * "
#    "FROM `footprintsrunning.twitter_scraping.trail_running`" # change to *your*` project ID, data set ID, and table name! 
#)

#results = gbqclient.query(query)

#print("Let's print it ugly.")
#print(list(results))

### Clean data

In [7]:
import pandas as pd
import re
import glob
import nltk
from nltk import word_tokenize, sent_tokenize, FreqDist
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords
import preprocessor as p
import numpy as np
words = set(nltk.corpus.words.words())
sw=set(stopwords.words('english'))

In [8]:
# Upload all local csv files to single pandas dataframe

path = 'data/' # use your path

all_files = glob.glob(os.path.join(path, "*.csv"))

tweets = pd.concat((pd.read_csv(f) for f in all_files), ignore_index=True)

In [9]:
def clean_tweet(tweet):
    if type(tweet) == float:
        return ""
    temp = tweet.lower()
    temp = re.sub("'", "", temp) # to avoid removing contractions in english
    temp = re.sub("@[A-Za-z0-9_]+","", temp)  # remove mentions
    temp = re.sub("#[A-Za-z0-9_]+","", temp)  # remove hashtags
    temp = re.sub(r'http\S+', '', temp)  # remove links
    temp = re.sub('[()!?]', ' ', temp)  # remove punct
    temp = re.sub('\[.*?\]',' ', temp)  # remove punct
    temp = re.sub("[^a-z0-9]"," ", temp) # remove non-alphanumeric
    temp = temp.split()  # tokenize
    temp = [w for w in temp if w not in sw]  # remove stopwords
    temp = " ".join(word for word in temp)  # rejoin into tweets again
    return temp

In [10]:
tweets['fulltext_clean'] = tweets['full_text'].apply(clean_tweet)  # create new column of cleaned tweets

In [11]:
tweets_final = tweets.drop('full_text', axis=1)  # drop orig full_text column

In [12]:
tweets['desc_clean'] = tweets['description'].apply(clean_tweet)  # create new column of cleaned tweets

In [13]:
tweets_final = tweets.drop('description', axis=1)  # drop orig full_text column

In [14]:
tweets_final.head()

,user_id,screen_name,location,following_count,following,followers_count,followers,totaltweets,date_created,tweet_id,retweetcount,full_text,fulltext_clean,desc_clean
0,848865692,kinggedcom,All over the World,0,<bound method User.friends of User(_api=<tweep...,350,<bound method User.followers of User(_api=<twe...,1617,2022-10-30 23:59:05+00:00,1586870335299031040,0,How to Get Paid to Run and 10 Best Companies &...,get paid run 10 best companies amp apps pay via,personal finance site kinglike content making ...
1,1277661655825088514,ephemeralisall,In a hive of scum and villainy,642,<bound method User.friends of User(_api=<tweep...,215,<bound method User.followers of User(_api=<twe...,7457,2022-10-30 23:57:49+00:00,1586870018494681089,1,"“Letting go gives us freedom, and freedom is t...",letting go gives us freedom freedom condition ...,trying make sense world creativity music films...
2,1341858024,WIHLiving,"Chattanooga, TN",6052,<bound method User.friends of User(_api=<tweep...,6171,<bound method User.followers of User(_api=<twe...,6574,2022-10-30 23:53:20+00:00,1586868890826686465,0,Listen to music while #running? Awesome @Spoti...,listen music awesome playlist,hi jan rc eat clean inspire living others simp...
3,3318927887,oldrunningmom,"Chicago, IL",2836,<bound method User.friends of User(_api=<tweep...,1964,<bound method User.followers of User(_api=<twe...,13152,2022-10-30 23:44:21+00:00,1586866630646288384,0,#WeeklyRunDown: Surviving Hell Week #OTF #OTFH...,surviving hell week via,outrunning old age rheumatoid arthritis wannab...
4,45187217,ToshioK,NaN,749,<bound method User.friends of User(_api=<tweep...,1889,<bound method User.followers of User(_api=<twe...,97661,2022-10-30 23:44:08+00:00,1586866574115627008,0,Autumn path. #Nature #Hiking #Trail #Forest #A...,autumn path,photographer lawyer traveler nowhere everywher...


### Upload data

### Note for JC: Need to update so table is not replaced each time, but instead code checks for duplicate tweet IDs and skips if ID is already in GBQ. Some combination of first checking for dupes and then if_exists="append" ??

In [16]:
pandas_gbq.to_gbq(tweets_final, table_id, gbq_proj_id, if_exists="replace")

100%|███████████████████████████████████████████| 1/1 [00:00<00:00, 2432.89it/s]


In [17]:
table = gbqclient.get_table(table_id)  # Make an API request.

# View table properties
print(
    "Got table '{}.{}.{}'.".format(table.project, table.dataset_id, table.table_id)
)
print("Table schema: {}".format(table.schema))
print("Table description: {}".format(table.description))
print("Table has {} rows".format(table.num_rows))

Got table 'footprintsrunning.twitter_scraping.trail_running'.
Table schema: [SchemaField('user_id', 'INTEGER', 'NULLABLE', None, (), None), SchemaField('screen_name', 'STRING', 'NULLABLE', None, (), None), SchemaField('location', 'STRING', 'NULLABLE', None, (), None), SchemaField('friends_count', 'INTEGER', 'NULLABLE', None, (), None), SchemaField('followers_count', 'INTEGER', 'NULLABLE', None, (), None), SchemaField('totaltweets', 'INTEGER', 'NULLABLE', None, (), None), SchemaField('date_created', 'STRING', 'NULLABLE', None, (), None), SchemaField('tweet_id', 'INTEGER', 'NULLABLE', None, (), None), SchemaField('retweetcount', 'INTEGER', 'NULLABLE', None, (), None), SchemaField('full_text', 'STRING', 'NULLABLE', None, (), None), SchemaField('fulltext_clean', 'STRING', 'NULLABLE', None, (), None), SchemaField('desc_clean', 'STRING', 'NULLABLE', None, (), None)]
Table description: None
Table has 89354 rows
